In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Gemini Pro 模型開始對話

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_chat.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise 標誌"><br>在 Colab Enterprise 中執行
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/gemini/getting-started/intro_gemini_chat.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Eric Dong](https://github.com/gericdong) |


## 概觀

這個筆記本示範如何使用 Python 和 LangChain 的 Vertex AI SDK 將對話提示傳送給 Gemini 1.0 Pro 模型 (`gemini-1.0-pro`)。Gemini 1.0 Pro 支援僅文字輸入的提示，包括自然語言任務、多輪文字和程式碼對話，以及程式碼產生。它可以輸出文字和程式碼。

深入了解 [傳送對話提示要求 (Gemini)](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/send-chat-prompts-gemini)。


### 目標

在本教學課程中，你將使用 Python 和 LangChain 的 Vertex AI SDK，瞭解如何將聊天提示訊息傳送到 Gemini 1.0 Pro 模型 (`gemini-1.0-pro`)。

你將完成下列任務：

- 使用 Python 的 Vertex AI SDK 傳送聊天提示訊息
- 使用 LangChain 傳送聊天提示訊息


### 成本
本教學課程使用 Google Cloud 的計費元件：

- Vertex AI

深入了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)並使用 [定價計算器](https://cloud.google.com/products/calculator/)，根據你的預計使用量產生成本估算。


## 開始使用


### 安裝函式庫


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 langchain-google-vertexai \
                                 langchain

### 重新啟動當前執行環境

若要在這個 Jupyter 執行環境中使用新安裝的套件，你必須重新啟動執行環境。你可以執行下方的單元格來達到這件事，這個單元格會重新啟動當前核心程序。

重新啟動可能會需要一分鐘以上。重新啟動完成後，請繼續進行下一個步驟。


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b> ⚠️ 在繼續之前，請等核心重新啟動完成。⚠️ </b>
</div>


### 驗證你的筆記本電腦環境 (僅限 Colab) 

如果你在 Google Colab 上執行此筆記本電腦，請執行以下Cell以驗證你的環境。

如果你使用 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，則不需要此步驟。


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 定義 Google Cloud 專案資訊及初始化 Vertex AI

針對你的專案初始化 Python 版的 Vertex AI SDK：


In [2]:
# Define project information
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### 匯入函式庫


In [3]:
from IPython.display import Markdown
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from vertexai.generative_models import Content, GenerativeModel, Part

## 使用 Python 版 Vertex AI SDK 發送對話提示

### 載入 Gemini 1.0 Pro 模型

Gemini 1.0 Pro 支援從文字提示產生文字和程式碼。


In [4]:
model = GenerativeModel("gemini-1.0-pro")

### 開始一段聊天對話

你開始一個有狀態的聊天對話，然後發送帶有配置參數的聊天詢問，包含文字生成配置和安全設定。


In [5]:
chat = model.start_chat()

response = chat.send_message(
    """You are an astronomer, knowledgeable about the solar system.
How many moons does Mars have? Tell me some fun facts about them.
"""
)

print(response.text)

Mars has two moons: Phobos and Deimos.

**Phobos**

* **Closest moon to Mars:** It orbits just 6,000 kilometers (3,700 miles) above the surface.
* **Unusual shape:** Phobos is not spherical but rather irregularly shaped, resembling a potato.
* **Astronomical sync:** Phobos orbits Mars three times a day, completing its orbit faster than any other known moon in the solar system.
* **Unique groove:** Phobos has a prominent groove or canyon called Stickney, which is thought to be the result of an impact.
* **Tidal acceleration:** Phobos is gradually spiraling inward towards Mars due to tidal forces. It is estimated that in about 100 million years, Phobos will either crash into Mars or break up into a ring.

**Deimos**

* **Outer moon of Mars:** Deimos orbits about 23,500 kilometers (14,600 miles) from the planet's surface.
* **Small and irregularly shaped:** Deimos is even smaller and more irregularly shaped than Phobos.
* **Slow orbit:** Deimos takes about 30 hours to complete one orbit a

你可以用`Markdown`來顯示生成的文本。


In [6]:
Markdown(response.text)

Mars has two moons: Phobos and Deimos.

**Phobos**

* **Closest moon to Mars:** It orbits just 6,000 kilometers (3,700 miles) above the surface.
* **Unusual shape:** Phobos is not spherical but rather irregularly shaped, resembling a potato.
* **Astronomical sync:** Phobos orbits Mars three times a day, completing its orbit faster than any other known moon in the solar system.
* **Unique groove:** Phobos has a prominent groove or canyon called Stickney, which is thought to be the result of an impact.
* **Tidal acceleration:** Phobos is gradually spiraling inward towards Mars due to tidal forces. It is estimated that in about 100 million years, Phobos will either crash into Mars or break up into a ring.

**Deimos**

* **Outer moon of Mars:** Deimos orbits about 23,500 kilometers (14,600 miles) from the planet's surface.
* **Small and irregularly shaped:** Deimos is even smaller and more irregularly shaped than Phobos.
* **Slow orbit:** Deimos takes about 30 hours to complete one orbit around Mars.
* **Captured asteroid:** Deimos is believed to be a captured asteroid rather than a moon that formed alongside Mars.
* **Speculation of being artificial:** In 1958, Soviet astronomer Iosif Shklovsky suggested that Deimos might be an artificial satellite placed in orbit around Mars by an advanced extraterrestrial civilization. However, this theory has not gained scientific acceptance.

你可以查看回應的元資料，包括 `safety_ratings` 和 `usage_metadata`。


In [7]:
print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "Mars has two moons: Phobos and Deimos.\n\n**Phobos**\n\n* **Closest moon to Mars:** It orbits just 6,000 kilometers (3,700 miles) above the surface.\n* **Unusual shape:** Phobos is not spherical but rather irregularly shaped, resembling a potato.\n* **Astronomical sync:** Phobos orbits Mars three times a day, completing its orbit faster than any other known moon in the solar system.\n* **Unique groove:** Phobos has a prominent groove or canyon called Stickney, which is thought to be the result of an impact.\n* **Tidal acceleration:** Phobos is gradually spiraling inward towards Mars due to tidal forces. It is estimated that in about 100 million years, Phobos will either crash into Mars or break up into a ring.\n\n**Deimos**\n\n* **Outer moon of Mars:** Deimos orbits about 23,500 kilometers (14,600 miles) from the planet\'s surface.\n* **Small and irregularly shaped:** Deimos is even smaller and more irregularly shaped 

你能夠擷取聊天會話的歷史紀錄。


In [8]:
print(chat.history)

[role: "user"
parts {
  text: "You are an astronomer, knowledgeable about the solar system.\nHow many moons does Mars have? Tell me some fun facts about them.\n"
}
, role: "model"
parts {
  text: "Mars has two moons: Phobos and Deimos.\n\n**Phobos**\n\n* **Closest moon to Mars:** It orbits just 6,000 kilometers (3,700 miles) above the surface.\n* **Unusual shape:** Phobos is not spherical but rather irregularly shaped, resembling a potato.\n* **Astronomical sync:** Phobos orbits Mars three times a day, completing its orbit faster than any other known moon in the solar system.\n* **Unique groove:** Phobos has a prominent groove or canyon called Stickney, which is thought to be the result of an impact.\n* **Tidal acceleration:** Phobos is gradually spiraling inward towards Mars due to tidal forces. It is estimated that in about 100 million years, Phobos will either crash into Mars or break up into a ring.\n\n**Deimos**\n\n* **Outer moon of Mars:** Deimos orbits about 23,500 kilometers (1

### 編碼聊天

Gemini 1.0 Pro 也支援根據文字提示生成程式碼。


In [9]:
code_chat = model.start_chat()

response = code_chat.send_message(
    "Write a function that checks if a year is a leap year"
)

print(response.text)

```python
def is_leap_year(year):
  """
  Checks if a year is a leap year.

  Args:
    year: The year to check.

  Returns:
    True if the year is a leap year, False otherwise.
  """

  if year % 4 != 0:
    return False

  if year % 100 == 0 and year % 400 != 0:
    return False

  return True
```


你可以在此多輪對話中生成單元測試來測試該功能。


In [10]:
response = code_chat.send_message("Write a unit test of the generated function")

print(response.text)

```python
import unittest

class TestIsLeapYear(unittest.TestCase):

    def test_is_leap_year(self):
        self.assertTrue(is_leap_year(2000))
        self.assertTrue(is_leap_year(2004))
        self.assertTrue(is_leap_year(2012))

    def test_is_not_leap_year(self):
        self.assertFalse(is_leap_year(1900))
        self.assertFalse(is_leap_year(1901))
        self.assertFalse(is_leap_year(2018))
```


### 加入聊天記錄

你可以透過加入角色「使用者」和「模型」的訊息，來加入聊天記錄。系統訊息可以設定在第一段中第一個訊息中。


In [11]:
chat2 = model.start_chat(
    history=[
        Content(
            role="user",
            parts=[
                Part.from_text(
                    """
    My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.
    Who do you work for?
    """
                )
            ],
        ),
        Content(role="model", parts=[Part.from_text("I work for Ned.")]),
        Content(role="user", parts=[Part.from_text("What do I like?")]),
        Content(role="model", parts=[Part.from_text("Ned likes watching movies.")]),
    ]
)

response = chat2.send_message("Are my favorite movies based on a book series?")
Markdown(response.text)

Yes, Lord of the Rings and Hobbit are based on the book series of the same names by J.R.R. Tolkien.

In [12]:
response = chat2.send_message("When were these books published?")
Markdown(response.text)

The Hobbit was published in 1937 and The Lord of the Rings was published in 1954.

## 使用 LangChain 傳送對話提示

Vertex AI Gemini API 與 LangChain Python SDK 整合後，可以更輕鬆地在 Gemini 模型之上建置應用程式。


### 開始聊天會話

你可以直接向 Gemini 1.0 Pro 模型傳送聊天提示訊息來開始聊天。Gemini 1.0 Pro 目前不支援 `SystemMessage`，但可以將 `SystemMessage` 設定為 `True` 來加入第一個人類訊息。


In [13]:
system_message = "You are a helpful assistant who translates English to French."
human_message = "Translate this sentence from English to French. I love programming."

messages = [SystemMessage(content=system_message), HumanMessage(content=human_message)]

chat = ChatVertexAI(
    model_name="gemini-1.0-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

result = chat.generate([messages])
print(result.generations[0][0].text)

J'aime la programmation.


你可以查看生成內容的元數據。


In [14]:
print(result.generations[0][0].generation_info)

{'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'citation_metadata': None}


### 使用帶聊天提示範本的聊天鏈


In [15]:
system_message = "You are a helpful assistant who translates English to French."
human_message = "Translate this sentence from English to French. I love programming."

messages = [SystemMessage(content=system_message), HumanMessage(content=human_message)]
prompt = ChatPromptTemplate.from_messages(messages)

chat = ChatVertexAI(
    model_name="gemini-1.0-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

chain = prompt | chat
chain.invoke({})

AIMessage(content="J'adore la programmation.")

### 使用對話鏈

你也可以在 `ConversationChain` 中封裝對話，其中內建的記憶體可記住使用者過去的輸入和模型的輸出。


In [16]:
model = ChatVertexAI(
    model_name="gemini-1.0-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant who is good at language translation."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="history", return_messages=True)
conversation = ConversationChain(llm=model, prompt=prompt, verbose=True, memory=memory)

conversation.invoke(
    input="Translate this sentence from English to French. I love programming."
)



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are a helpful assistant who is good at language translation.
Human: Translate this sentence from English to French. I love programming.

> Finished chain.


{'input': 'Translate this sentence from English to French. I love programming.',
 'history': [HumanMessage(content='Translate this sentence from English to French. I love programming.'),
  AIMessage(content=" J'adore la programmation.")],
 'response': " J'adore la programmation."}

In [17]:
conversation.invoke("Translate it to Spanish")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are a helpful assistant who is good at language translation.
Human: Translate this sentence from English to French. I love programming.
AI:  J'adore la programmation.
Human: Translate it to Spanish

> Finished chain.


{'input': 'Translate it to Spanish',
 'history': [HumanMessage(content='Translate this sentence from English to French. I love programming.'),
  AIMessage(content=" J'adore la programmation."),
  HumanMessage(content='Translate it to Spanish'),
  AIMessage(content='Me encanta programar.')],
 'response': 'Me encanta programar.'}